In [ ]:
# group by node id (since rows are currently node events) .mean of is_crash to have dummy danger metric
# associate the node ids with user traslation table to have node ID with tract ID then associate with demographic data
# linear regression: ex: percentage black with crash ratio

In [126]:
import pandas as pd

In [127]:
# load node events
node_events_df = pd.read_csv("../../data/unified_dataset.csv")

KeyboardInterrupt: 

In [ ]:
# check shape
node_events_df.shape

In [ ]:
# examine head
node_events_df.head()

In [ ]:
# check columns
node_events_df.columns

In [ ]:
# create df with node ID and IS_CRASH
node_ids_crash_avg = node_events_df[['NODE_ID','IS_CRASH']]
node_ids_crash_avg.head()

In [ ]:
# group by node ID and take average of IS_CRASH to have toy danger metric
node_id_crash_df = node_ids_crash_avg.groupby(by='NODE_ID').agg('mean')

In [ ]:
# examine head
node_id_crash_df.head()

In [ ]:
# check unique NOIDE_ID in df matches number of rows in node_id_crash_df
assert len(set(node_events_df['NODE_ID'])) == node_id_crash_df.shape[0]

# check how many unique NODE_ID in original df
len(set(node_events_df['NODE_ID']))

In [ ]:
# load node id to census tract key table
node_id_census_tract = pd.read_csv("../../data/node_id_census_tract_key.csv")

# check shape
node_id_census_tract.shape

In [ ]:
# examine head
node_id_census_tract.head()

In [ ]:
# look at why there are more rows in census 
# ASSUMING THESE ARE NODES FOR WHICH WE HAVE NO NODE EVENTS (neither crashes nor citiBike)
print("number of node ids in census tract df not in crash df: ", len(set(node_id_census_tract['osmid'])) - len(set(node_events_df['NODE_ID'])))

node_id_crash_df.columns

In [ ]:
# merge on node ids: census tracts and avg crash
node_ids__crash_avg_census_tracts = node_id_crash_df.merge(node_id_census_tract, how = 'left', left_index=True, right_on='osmid')

# rename columns to be clearer
col_names = {'osmid':'NODE_ID','IS_CRASH':'CRASH_AVG','ct2010':'CENSUS_TRACT_ID'}
node_ids__crash_avg_census_tracts.rename(columns=col_names, inplace=True)

# check shape
node_ids__crash_avg_census_tracts.shape

In [ ]:
# examine head
node_ids__crash_avg_census_tracts.head()

In [ ]:
node_ids__crash_avg_census_tracts.sort_values('CRASH_AVG', ascending=False)

In [ ]:
print("Nodes with no Crashes: number of nodes (# CRASH / # EVENTS) == 0 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']==0))
# there are Crash Averages ==1 so looking into this:
## do we want to remove Crash Averages that are above a certain fraction?
## for example, greater than .001 meaning that there was one crash in 1000 events for the node

print("number of nodes (# CRASH / # EVENTS) == 1 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']==1))
print("number of nodes (# CRASH / # EVENTS) > 1 / 10 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.1))
print("number of nodes (# CRASH / # EVENTS) > 1 / 100 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.01))
print("number of nodes (# CRASH / # EVENTS) > 1 / 1000 : ", sum(node_ids__crash_avg_census_tracts['CRASH_AVG']>.001))

In [ ]:
# look at sample CRASH EVENT == 1 node from original df
node_events_df.loc[node_events_df['NODE_ID']==1728266963,:]

In [ ]:
# look at rows with Crash Avg == 1 to ignore in correlation
crash_avg_equals_one = node_ids__crash_avg_census_tracts.loc[node_ids__crash_avg_census_tracts['CRASH_AVG']==1,:]

In [ ]:
# create df with rows where Crash Avg < 1
node_ids__crash_avg_census_tracts_clean = node_ids__crash_avg_census_tracts.loc[node_ids__crash_avg_census_tracts['CRASH_AVG']<1,:]

# check shape
node_ids__crash_avg_census_tracts_clean.shape

In [ ]:
# save csv with node ids, census tracts, and DUMMY DANGER metric
node_ids__crash_avg_census_tracts_clean.to_csv(path_or_buf='../../data/DUMMY_CRASH_AVG_FOR_CORRELATION_TESTS_node_id_census_tract_key.csv')